In [859]:
import sys

sys.path.append("./../..")

from numba import jit
from numba.types import bool_, int_
import numpy as np
import matplotlib.pyplot as plt
from InternalLibrary.StatisticalFunctions import stat_corr_single
from scipy.optimize import curve_fit
from scipy.stats import moment

from tqdm import tqdm

from sbi import analysis as analysis
from sbi import utils as utils
from sbi.inference import SNPE, simulate_for_sbi, infer
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
import torch

# Run and Tumble Model

In [860]:
@jit
def RunTumble(param, n_sim, dt=0.01, TotalT=1, alpha=0.5):
    k = param[0]
    omega = param[1]
    D = omega**2 / alpha
    
    total_time_steps = np.int64(TotalT / dt)
    
    x = np.zeros((1, n_sim))
    x_trace = np.zeros((n_sim, total_time_steps))
    
    direction_trace = np.ones((n_sim, total_time_steps))
    direction = np.ones((1, n_sim))
    
    for n in range(1, total_time_steps):
        change = np.random.uniform(0, 1, size=(1, n_sim)) < alpha * dt / 2 # Change direction with probability alpha*dt/2
        change = -(change *2 -1) # From boolean to -1 or 1 (false = 1, true = -1)
        direction = change*direction # Change direction if change is true
        x[:] = ( x[:] + (-k * x[:] + omega * direction) * dt + np.sqrt(2 * D * dt) * np.random.normal(0, 1, size=(1, n_sim))) 
        
        x_trace[:, n] = x[:,]
        direction_trace[:, n] = direction[:,]
    
    return x_trace, direction_trace           

In [861]:
param = [1, 1]
n_sim = 200
dt = 1e-6
TotalT = 2
alpha = 0.5
t = np.arange(0, TotalT, dt)
omega = param[1]
k = param[0]

In [862]:
x_trace, direction_trace = RunTumble(param, n_sim, dt, TotalT, alpha)